In [1]:
import json
from pathlib import Path
from napthaville_environment.maze import Maze
from naptha_sdk.storage.storage_client import StorageClient
from naptha_sdk.schemas import NodeConfigUser

In [2]:
from napthaville_environment.utils import read_file_to_list
from napthaville_environment.schemas import (
    MazeConfig, 
    MazeState, 
    TileDetails, 
    TileLocation, 
    PixelCoordinate, 
    TileLevel,
    TilePath,
    VisionRadius,
    NearbyTiles
)
from naptha_sdk.storage.schemas import (
    StorageType,
    StorageLocation,
    StorageObject,
    CreateStorageRequest,
    ReadStorageRequest,
    UpdateStorageRequest,
    DeleteStorageRequest, 
    DatabaseReadOptions,
    ListStorageRequest
)

In [3]:
config = json.load(open("/Users/arshath/play/napthaville_environment/napthaville_environment/configs/deployment.json"))
config = config[0]["config"]
node = NodeConfigUser(
    ip = "localhost",
    user_communication_port=7001,
    user_communication_protocol="http",
)
storage_client = StorageClient(node)

2025-02-20 21:17:21,085 - naptha_sdk.storage.storage_client - INFO - Storage Provider URL: http://localhost:7001


In [4]:
# maze = await Maze.create(config[0]["config"], storage_client)

In [5]:
delete_request = DeleteStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment"
)
await storage_client.execute(delete_request)


True

In [6]:
# Basic maze config
maze_name = config["world_name"]    
maze_width = config["maze_width"]
maze_height = config["maze_height"]
sq_tile_size = config["sq_tile_size"]
special_constraint = config["special_constraint"]
env_matrix_path = config["env_matrix_path"]

# Database config
storage_config = config["storage_config"]
schema = storage_config["path"]
storage_schema = storage_config["storage_schema"]

# Initialize empty placeholders
collision_maze = []
tiles = []
address_tiles = dict()

<!-- ### is_initialized -->

In [7]:
# # Check if napthaville_environment table exists and check each type
# table_types = ['tiles', 'collision_maze', 'address_tiles']

# for data_type in table_types:
#     check_request = ReadStorageRequest(
#         storage_type=StorageType.DATABASE,
#         path="napthaville_environment",
#         options=DatabaseReadOptions(
#             columns=["data"],
#             conditions=[{
#                 "field": "type",
#                 "op": "=",
#                 "value": data_type
#             }]
#         ).model_dump()
#     )

#     try:
#         check_result = await storage_client.execute(check_request)
#         print(f"{data_type} data exists in table")
#     except Exception as e:
#         print(f"Check for {data_type} failed: {str(e)}")

### load env matrix

In [8]:
env_matrix_path = Path("/Users/arshath/play/napthaville_environment/napthaville_environment/data/matrix")
blocks_folder = env_matrix_path / "special_blocks"

# Load blocks
_wb = blocks_folder / "world_blocks.csv"
wb_rows = read_file_to_list(_wb, header=False)
wb = wb_rows[0][-1]

_sb = blocks_folder / "sector_blocks.csv" 
sb_rows = read_file_to_list(_sb, header=False)
sb_dict = dict()
for i in sb_rows: sb_dict[i[0]] = i[-1]

_ab = blocks_folder / "arena_blocks.csv"
ab_rows = read_file_to_list(_ab, header=False)
ab_dict = dict()
for i in ab_rows: ab_dict[i[0]] = i[-1]

_gob = blocks_folder / "game_object_blocks.csv"
gob_rows = read_file_to_list(_gob, header=False)
gob_dict = dict()
for i in gob_rows: gob_dict[i[0]] = i[-1]

_slb = blocks_folder / "spawning_location_blocks.csv"
slb_rows = read_file_to_list(_slb, header=False)
slb_dict = dict()
for i in slb_rows: slb_dict[i[0]] = i[-1]

# Load mazes
maze_folder = env_matrix_path / "maze"
_cm = maze_folder / "collision_maze.csv"
collision_maze_raw = read_file_to_list(_cm, header=False)[0]
_sm = maze_folder / "sector_maze.csv"
sector_maze_raw = read_file_to_list(_sm, header=False)[0]
_am = maze_folder / "arena_maze.csv"
arena_maze_raw = read_file_to_list(_am, header=False)[0]
_gom = maze_folder / "game_object_maze.csv"
game_object_maze_raw = read_file_to_list(_gom, header=False)[0]
_slm = maze_folder / "spawning_location_maze.csv"
spawning_location_maze_raw = read_file_to_list(_slm, header=False)[0]

# Convert 1D to 2D
collision_maze = []
sector_maze = []
arena_maze = []
game_object_maze = []
spawning_location_maze = []
for i in range(0, len(collision_maze_raw), maze_width): 
    tw = maze_width
    collision_maze += [collision_maze_raw[i:i+tw]]
    sector_maze += [sector_maze_raw[i:i+tw]]
    arena_maze += [arena_maze_raw[i:i+tw]]
    game_object_maze += [game_object_maze_raw[i:i+tw]]
    spawning_location_maze += [spawning_location_maze_raw[i:i+tw]]

# Initialize tiles
tiles = []
for i in range(maze_height): 
    row = []
    for j in range(maze_width):
        tile_details = dict()
        tile_details["world"] = wb
        
        tile_details["sector"] = ""
        if sector_maze[i][j] in sb_dict: 
            tile_details["sector"] = sb_dict[sector_maze[i][j]]
        
        tile_details["arena"] = ""
        if arena_maze[i][j] in ab_dict: 
            tile_details["arena"] = ab_dict[arena_maze[i][j]]
        
        tile_details["game_object"] = ""
        if game_object_maze[i][j] in gob_dict: 
            tile_details["game_object"] = gob_dict[game_object_maze[i][j]]
        
        tile_details["spawning_location"] = ""
        if spawning_location_maze[i][j] in slb_dict: 
            tile_details["spawning_location"] = slb_dict[spawning_location_maze[i][j]]
        
        tile_details["collision"] = False
        if collision_maze[i][j] != "0": 
            tile_details["collision"] = True

        tile_details["events"] = set()
        
        row += [tile_details]
    tiles += [row]

# Initialize game object events
for i in range(maze_height):
    for j in range(maze_width): 
        if tiles[i][j]["game_object"]:
            object_name = ":".join([tiles[i][j]["world"], 
                                tiles[i][j]["sector"], 
                                tiles[i][j]["arena"], 
                                tiles[i][j]["game_object"]])
            go_event = (object_name, None, None, None)
            tiles[i][j]["events"].add(go_event)

# Initialize address tiles
address_tiles = dict()
for i in range(maze_height):
    for j in range(maze_width): 
        addresses = []
        if tiles[i][j]["sector"]: 
            add = f'{tiles[i][j]["world"]}:'
            add += f'{tiles[i][j]["sector"]}'
            addresses += [add]
        if tiles[i][j]["arena"]: 
            add = f'{tiles[i][j]["world"]}:'
            add += f'{tiles[i][j]["sector"]}:'
            add += f'{tiles[i][j]["arena"]}'
            addresses += [add]
        if tiles[i][j]["game_object"]: 
            add = f'{tiles[i][j]["world"]}:'
            add += f'{tiles[i][j]["sector"]}:'
            add += f'{tiles[i][j]["arena"]}:'
            add += f'{tiles[i][j]["game_object"]}'
            addresses += [add]
        if tiles[i][j]["spawning_location"]: 
            add = f'<spawn_loc>{tiles[i][j]["spawning_location"]}'
            addresses += [add]

        for add in addresses: 
            if add in address_tiles: 
                address_tiles[add].add((j, i))
            else: 
                address_tiles[add] = set([(j, i)])

In [9]:
storage_schema = config["storage_config"]["storage_schema"]

# Create napthaville_environment table
table_name = "napthaville_environment"
schema_def = storage_schema["napthaville_environment"]

create_table_request = CreateStorageRequest(
    storage_type=StorageType.DATABASE,
    path=table_name,
    data={"schema": schema_def}
)

try:
    await storage_client.execute(create_table_request)
    print(f"Created table {table_name}")
except Exception as e:
    print(f"Error creating table {table_name}: {str(e)}")

Created table napthaville_environment


In [10]:
# Try simplified but complete tiles data
full_tiles_data = {
    "maze_name": maze_name,
    "tiles_matrix": [[{
        "world": tile["world"],
        "sector": tile["sector"],
        "arena": tile["arena"],
        "game_object": tile["game_object"],
        "spawning_location": tile["spawning_location"],
        "collision": tile["collision"],
        "events": []  # Start with empty list instead of set
    } for tile in row] for row in tiles]
}

tiles_create = CreateStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    data={"data": {
        "type": "tiles",
        "data": json.dumps(full_tiles_data)
    }}
)

try:
    await storage_client.execute(tiles_create)
    print("Full tiles data inserted successfully")
except Exception as e:
    print(f"Error inserting tiles data: {str(e)}")

Full tiles data inserted successfully


In [11]:
# Collision maze data
collision_data = {
    "maze_name": maze_name,
    "collision_matrix": collision_maze
}

collision_create = CreateStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    data={"data": {
        "type": "collision_maze",
        "data": json.dumps(collision_data)
    }}
)

try:
    r = await storage_client.execute(collision_create)
    print("Collision maze data inserted successfully")
except Exception as e:
    print(f"Error inserting collision maze data: {str(e)}")

Collision maze data inserted successfully


In [12]:
# Address tiles data
# Convert sets to lists for JSON serialization
address_data = {
    "maze_name": maze_name,
    "address_mappings": {
        addr: list(coords) for addr, coords in address_tiles.items()
    }
}

address_create = CreateStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    data={"data": {
        "type": "address_tiles",
        "data": json.dumps(address_data)
    }}
)

try:
    await storage_client.execute(address_create)
    print("Address tiles data inserted successfully")
except Exception as e:
    print(f"Error inserting address tiles data: {str(e)}")

Address tiles data inserted successfully


In [13]:
request = ReadStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    options= {"condition": {"type": "tiles"}}

)
result = await storage_client.execute(request)

# Parse the JSONB data
data = json.loads(result.data[0]["data"])  # Note the nested "data" access
tiles_matrix = data["tiles_matrix"]

# Convert event lists back to sets
for i in range(len(tiles_matrix)):
    for j in range(len(tiles_matrix[i])):
        if tiles_matrix[i][j]["events"]:
            tiles_matrix[i][j]["events"] = set(tuple(e) for e in tiles_matrix[i][j]["events"])

In [14]:
request = ReadStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    options= {"conditions": [{"type": "collision_maze"}]}
)
result = await storage_client.execute(request)


data = json.loads(result.data[0]["data"])
collision_matrix = data["collision_matrix"]


In [15]:
request = ReadStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    options= {"conditions": [{"type": "address_tiles"}]}
)
result = await storage_client.execute(request)

data = json.loads(result.data[0]["data"])
address_mappings = data["address_mappings"]

In [16]:
event_to_add = ('double studio:double studio:bedroom 2:bed', None, None, None)

event = tiles_matrix[1][1]['events']
event = set(tuple(e) for e in event)
event.add(event_to_add)
tiles_matrix[1][1]['events'] = list(event)

In [17]:
# make it into tile_matrix

tiles_matrix = {"tiles_matrix": [[{
    "world": tile["world"],
    "sector": tile["sector"],
    "arena": tile["arena"],
    "game_object": tile["game_object"],
    "spawning_location": tile["spawning_location"],
    "collision": tile["collision"],
    "events": list(tile["events"])
} for tile in row] for row in tiles_matrix]}


In [19]:
tiles_matrix["tiles_matrix"][1][1]['events']

[('double studio:double studio:bedroom 2:bed', None, None, None)]

In [20]:
update_request = UpdateStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    data={"data": {
        "data": json.dumps(tiles_matrix)
    }},
    options={"condition": {"type": "tiles"}}
)

await storage_client.execute(update_request)

StorageObject(location=StorageLocation(storage_type=<StorageType.DATABASE: 'db'>, path='napthaville_environment', options={}), data={'location': {'storage_type': 'db', 'path': 'napthaville_environment', 'options': {}}, 'data': 1, 'metadata': {'content_type': None, 'created_at': None, 'modified_at': None, 'size': None, 'checksum': None, 'tags': {}, 'custom': {}}}, metadata=StorageMetadata(content_type=None, created_at=None, modified_at=None, size=None, checksum=None, tags={}, custom={}))

In [21]:
read_request = ReadStorageRequest(
    storage_type=StorageType.DATABASE,
    path="napthaville_environment",
    options={"conditions": [{"type": "tiles"}]}
)
result = await storage_client.execute(read_request)


In [22]:
data = json.loads(result.data[0]["data"])
tiles_matrix = data["tiles_matrix"]

# Convert event lists back to sets
for i in range(len(tiles_matrix)):
    for j in range(len(tiles_matrix[i])):
        if tiles_matrix[i][j]["events"]:
            tiles_matrix[i][j]["events"] = set(tuple(e) for e in tiles_matrix[i][j]["events"])
            

In [23]:
tiles_matrix[1][1]['events']

{('double studio:double studio:bedroom 2:bed', None, None, None)}